In [1]:
import numpy as np
from tqdm.notebook import tqdm
import csv
import networkx as nx
import random
import pickle
import pandas as pd
from itertools import groupby
from random import shuffle

import sys
sys.path.append('../../')
from utils import *

In [2]:
#dataset = "pr_school";gap = 300;state_gap = 1200;nb_states_L = 3;nb_states_e = 8
#dataset = "h_school11";gap = 1200;state_gap = 1200;nb_states_L = 3;nb_states_e = 7
#dataset = "h_school13";gap = 1200;state_gap = 1200;nb_states_L = 4;nb_states_e = 29
#dataset = "InVS15"; gap = 3600; state_gap = 3600; nb_states_L = 3;nb_states_e = 12
dataset = "hypertext";gap = 1200;state_gap = 1200;nb_states_L = 3;nb_states_e = 20

In [3]:
path_gen = '../../Generated_graphs/d2/'

# Functions

In [4]:
def upload_edgelist_D(dataset,gap):

    edgelist = pd.read_csv("../../Datasets/"+dataset+".dat",sep=" ",names=["t","i","j"])
    nb_nodes = max(max(edgelist.i),max(edgelist.j)) + 1
    nodes_list = np.arange(0,nb_nodes)
    times = [int(x/gap) for x in edgelist.t]
    edgelist.t = times

    return nodes_list,edgelist

In [5]:
def upload_edgelist_surrogate(dataset,net,net_idx,nb_states_e,gap,nb_states_L):
    path = '../../Generated_graphs/d2/'
    if net == 'E':
        directory = path+"%s_E_gap_%d_%d_states_e/"%(dataset,gap,nb_states_e)
        edgelist = pd.read_csv(directory+"/%d.dat"%net_idx,sep=" ",names=["t","i","j"])

    elif net == 'ES':       
        directory = path+"%s_ES_gap_%d_%d_states_e_%d_states_L/"%(dataset,gap,nb_states_e,nb_states_L)
        edgelist = pd.read_csv(directory+"/%d.dat"%net_idx,sep=" ",names=["t","i","j"])

    elif net == 'EST':       
        directory = path+"%s_EST_gap_%d_%d_states_e_%d_states_L/"%(dataset,gap,nb_states_e,nb_states_L)
        edgelist = pd.read_csv(directory+"/%d.dat"%net_idx,sep=" ",names=["t","i","j"])
    
    nb_nodes = max(max(edgelist.i),max(edgelist.j)) + 1
    nodes_list = np.arange(0,nb_nodes)
    times = [int(x/gap) for x in edgelist.t]
    edgelist.t = times

    return nodes_list,edgelist

In [6]:
def obtain_edgelist_dict(data_):
    edgelist_t_ = {}
    for t in range(max(data_.t) + 1):
        edgelist_t_[t] = []
    for _, row in data_.iterrows():
        if row['t'] in edgelist_t_:
            if [row['i'],row['j']] not in edgelist_t_[row['t']]:
                edgelist_t_[row['t']].append([row['i'],row['j']])
            if row['i'] > row['j']:
                print('error1',row['i'],row['j'])
        else:
            print('error2')
    return edgelist_t_

In [7]:
def equalLists(lists):
    return not lists or all(lists[0] == b for b in lists[1:])

In [8]:
def evaluate_inventory(Inv_dict):
    inv_A = 0
    inv_B = 0
    inv_AB = 0
    for el in Inv_dict:
        if Inv_dict[el] == ['A']:
            inv_A += 1
        elif Inv_dict[el] == ['B']:
            inv_B += 1
        elif Inv_dict[el] == ['A','B'] or Inv_dict[el] == ['B','A']:
            inv_AB += 1
        else:
            print('error', Inv_dict[el])
    return inv_A, inv_B, inv_AB

In [18]:
def simulate(eta, t0, edgelist_t, nodes_list):
    
    #Initialize time 0
    x = list(np.random.uniform(low=0, high=1, size=len(nodes_list)))
    Inv = {} # inventory
    for node in nodes_list:
        if x[node] <= 0.5:
            Inv[node] = ['A']
        else:
            Inv[node] = ['B']
    t = t0
    t_conv = 0
    m = 0
    for times in range(1000000):
        shuffle(edgelist_t[t])
        for edge in edgelist_t[t]:
            shuffle(edge)
            speaker = edge[0]
            hearer = edge[1]
            if Inv[speaker] != Inv[hearer] or len(Inv[speaker]) == 2:
                name = random.choice(Inv[speaker])
                if name in Inv[hearer]:
                    r = random.uniform(0, 1)
                    if r < eta:
                        Inv[speaker] = [name]
                        Inv[hearer] = [name]
                else:
                    Inv[hearer].append(name)
        if not times%10:
            inv_A, inv_B, inv_AB = evaluate_inventory(Inv)

        if max(inv_A, inv_B, inv_AB) > len(nodes_list)*98/100: # stabilized
            break
        if equalLists(list(Inv.values())): # stabilized
            break 
            
        t += 1
        if t > max(edgelist_t.keys()): # repeat temporal network if necessary
            t = 0 
        t_conv += 1

    winner = [inv_A, inv_B, inv_AB].index(max(inv_A, inv_B, inv_AB))
    if winner == 0:
        final_name = 'A'
    elif winner == 1:
        final_name = 'B'
    elif winner == 2:
        final_name = 'AB'
    return t_conv, final_name

# Simulate

In [10]:
folder = 'results/' # to save results
if not os.path.exists(folder):
    os.makedirs(folder)

In [11]:
nb_runs = 10

In [12]:
eta_range = np.linspace(0.4,1,num=7)
eta_range


array([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

D

In [14]:
net = 'D'

In [19]:
# Edgelist
nodes_list,data = upload_edgelist_D(dataset,gap)
edgelist_t = obtain_edgelist_dict(data)

non_empty_t = np.unique(data.t)

for eta in eta_range:
    filename = "results/%s_%s_eta_%.2f.txt"%(dataset,net,eta)
    file = open(filename, 'a+')
    for runs in range(nb_runs):
        t0 = random.choice(non_empty_t)
        t_conv, final_name = simulate(eta, t0, edgelist_t, nodes_list)

        file.write('%d %s\n'%(t_conv, final_name))

    file.close()

surrogates

In [20]:
for net in ['E','ES','EST']:
    print(net)
    for net_idx in tqdm(range(10)):
        # Edgelist 
        nodes_list,data = upload_edgelist_surrogate(dataset,net,net_idx,nb_states_e,gap,nb_states_L)
        edgelist_t = obtain_edgelist_dict(data)
        
        non_empty_t = np.unique(data.t)

        for eta in eta_range:
            filename = "results/%s_%s_eta_%.2f.txt"%(dataset,net,eta)
            file = open(filename, 'a+')
            for runs in range(nb_runs):
                t0 = random.choice(non_empty_t)
                t_conv, final_name = simulate(eta, t0, edgelist_t, nodes_list)
                file.write('%d %s\n'%(t_conv, final_name))
            file.close()

E


  0%|          | 0/10 [00:00<?, ?it/s]

net_idx 0
net_idx 1
net_idx 2
net_idx 3
net_idx 4
net_idx 5
net_idx 6
net_idx 7
net_idx 8
net_idx 9
ES


  0%|          | 0/10 [00:00<?, ?it/s]

net_idx 0
net_idx 1
net_idx 2
net_idx 3
net_idx 4
net_idx 5
net_idx 6
net_idx 7
net_idx 8
net_idx 9
EST


  0%|          | 0/10 [00:00<?, ?it/s]

net_idx 0
net_idx 1
net_idx 2
net_idx 3
net_idx 4
net_idx 5
net_idx 6
net_idx 7
net_idx 8
net_idx 9
